In [1]:
import tweepy
import pandas as pd
from textblob import TextBlob

consumer_key = "clPBLLDHG5mTS5ffzAzaoNRm1"
consumer_secret = "scBlLPRjI4xtUfhgBu6joELfJOeOMxTI19CdqvrlXmmBEVu209"

access_key= "962177014311432192-CoXC8gXSe3jZPqLfmOz1W0nTg5PS6TI"
access_key_secret = "8VAR6phmtnzB3NVCbU8yKWleE8s5ZivQOqEZaH9OEVako"

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key,access_key_secret)
api=tweepy.API(auth)

result = []

for tweet in tweepy.Cursor(api.search,q='%23cryptocurrency',since='2018-05-20` 11:00:00').items(200):
    result.append(tweet)
    
print(len(result))

# Converting the JSON data to a Pandas dataframe

def todf(tweets):
    dataset = pd.DataFrame() #Empty dataframe
    
    dataset['tweet id']= [tweet.id for tweet in tweets]
    dataset['tweet_text'] = [tweet.text for tweet in tweets]
    dataset['Time'] = [tweet.created_at for tweet in tweets]
    dataset['userTimezone'] = [tweet.user.time_zone for tweet 
    in tweets]
    return dataset

dataset = todf(result)
print(dataset.info())
print(dataset['Time'])

dataset.to_csv('New_file.csv')

# This program will generate tweets for the date mentioned as the "since" argument in tweepy.cursor()
# Using this program, tweets from 15th April to 12th May have been collected in the "combined.csv" file
# Everytime this netbook runs, a new file named " New_file.csv" will be created in the directory.



111
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
tweet id        111 non-null int64
tweet_text      111 non-null object
Time            111 non-null datetime64[ns]
userTimezone    0 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.5+ KB
None
0     2018-07-20 07:59:45
1     2018-07-20 07:00:03
2     2018-07-20 02:00:06
3     2018-07-19 12:00:08
4     2018-07-19 09:00:07
5     2018-07-18 19:50:36
6     2018-07-18 15:30:06
7     2018-07-18 15:21:05
8     2018-07-18 14:15:42
9     2018-07-18 12:00:07
10    2018-07-18 09:00:07
11    2018-07-18 07:01:25
12    2018-07-17 12:00:08
13    2018-07-17 11:53:27
14    2018-07-17 11:04:53
15    2018-07-17 09:00:03
16    2018-07-17 00:30:06
17    2018-07-16 23:00:11
18    2018-07-16 12:00:08
19    2018-07-16 09:00:09
20    2018-07-16 08:00:11
21    2018-07-15 12:00:07
22    2018-07-15 09:00:02
23    2018-07-14 12:00:07
24    2018-07-14 10:50:28
25    2018-07-14 10

### Bitcoin Prices : Data Collection and Wrangling

Bitcoin Prices from April 11th, 2018 to May 11th, 2018 were collected on an hourly basis from https://www.coindesk.com/price/ and saved as "price" dataframe in "Bitcoin_Prices_April 11_May 11.csv" file. The prices are then classified into two classes as following:

P(t) - P(t+1) > 0   : Price Difference Class = 1
P(t) - P(t+1) <= 0  : Price Difference Class = 0

Based on the classification, a new column label " Price Difference Class" is added to "price" dataframe.


In [2]:
import pandas as pd
from datetime import datetime

price = pd.read_csv('Bitcoin_Prices_April 11_May 11.csv',parse_dates=True)

price.drop([744,745],inplace=True)

delta_price = []

for i in range(1,len(price.index)):
    
     delta_price.append(price['Close Price'].loc[i] - price['Close Price'].loc[i-1])


delta_price.append(0)

delta_price_bin = []


for j in delta_price:
    
    if j <=0:
        delta_price_bin.append(0)
    else:
        delta_price_bin.append(1)


price['Price Difference Class'] = delta_price_bin

print(price.tail())

price['Date'] = price['Date'].astype('datetime64[s]')

price.index = pd.to_datetime(price['Date'])

price.index = price.index.round('1H')

price.to_csv('Bitcoin Prices.csv')


                    Date  Close Price  Price Difference Class
739  2018-05-11 19:00:00      8601.10                       0
740  2018-05-11 20:00:00      8436.20                       0
741  2018-05-11 21:00:00      8392.86                       1
742  2018-05-11 22:00:00      8426.23                       0
743  2018-05-11 23:00:00      8392.00                       0


### Visualizing Sentiments of Tweets and Bitcoin Prices

price dataframe converted to DateTime Index with time rounded of to closest 1 hour
tweet dataframe converted to DateTime Index with time rounded pf to closest 1 hour. A new column label "Sentiment" ,reflecting the polarity of the tweet, is added to the dataframe. 

df = tweet merged with price dataframe. DateTime index with common time indices of price and tweet dataframes  

In [3]:
import pandas as pd
from textblob import TextBlob
from datetime import datetime,timedelta
import matplotlib.pyplot as plt


tweet = pd.read_csv('combined.csv',parse_dates=True)
price = pd.read_csv('Bitcoin Prices.csv')


tweet['Time'] = tweet['Time'].astype('datetime64[s]')

tweet.index = pd.to_datetime(tweet['Time']- timedelta(hours=1))
tweet.index = tweet.index.round('1H')

price['Date'] = price['Date'].astype('datetime64[s]')

price.index = pd.to_datetime(price['Date'])

price.index = price.index.round('1H')

senti = []

for i in tweet['tweet_text']:
    
    
    senti.append(TextBlob(i).polarity)
    

tweet['Sentiment'] = senti


count_0 = 0
count_pos = 0
count_neg =0

for j in senti:
    
    if j==0:
        count_0+=1
        
    elif j>0:
        
    
        count_pos+=1
    else :
        
        count_neg+=1


print('\n Neutral Sentiment :' + str(count_0))

print('\n Positive Sentiment :' + str(count_pos))

print('\n Negative Sentiment :' + str(count_neg))

senti_list = [count_0,count_pos,count_neg]

df = tweet.merge(price,left_index=True,right_index=True,how='inner')

df.drop(['Date','Date.1','Time'],axis=1,inplace=True)

print(df.head())

print(df.info())

print("\n Following are some tweets with postive polarity of sentiment :")

if df['Sentiment'].any() > 0 : 
    print(df['tweet_text'].head())
    
    


 Neutral Sentiment :302

 Positive Sentiment :164

 Negative Sentiment :23
                         tweet id  \
2018-05-03 01:00:00  9.920000e+17   
2018-05-03 01:00:00  9.920000e+17   
2018-05-03 03:00:00  9.920000e+17   
2018-05-03 04:00:00  9.920000e+17   
2018-05-03 05:00:00  9.920000e+17   

                                                            tweet_text  \
2018-05-03 01:00:00  BOUGHT [ #XRPBTC | #binance | Price: 0.0000927...   
2018-05-03 01:00:00  RT @litenettcom: With an audience of 10 millio...   
2018-05-03 03:00:00  #Cryptos: \r\n\r\n#BTC 9270.10$ | 7754.53€\r\n...   
2018-05-03 04:00:00  RT @tihosay: Tihosay Pre-ICO will begin April ...   
2018-05-03 05:00:00  #Cryptos: \r\n\r\n#BTC 9252.40$ | 7739.73€\r\n...   

                     Sentiment  Close Price  Price Difference Class  
2018-05-03 01:00:00        0.0      9221.87                       1  
2018-05-03 01:00:00        0.0      9221.87                       1  
2018-05-03 03:00:00        0.0      9222.94   